In [1]:
%load_ext autoreload
%autoreload 2
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import hist

import sys
sys.path.append("../..")
from analyzer.dumperReader.reader import *

In [2]:
reader = DumperReader("/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v1/dumper_101.root") #"../runs_results/histo.root:ticlDumper")

In [9]:
allTracksters = reader.tracksters_zipped[["ts_id", "raw_pt", "raw_energy", "barycenter_eta"]]
allTracksters

<Array [[{ts_id: 0, ...}, ..., {...}], ...] type='100 * var * {ts_id: int64...'>

In [12]:
tracksters = tracksters_toDf(tracksters_getSeeds(allTracksters))
tracksters


raw_pt  raw_energy  barycenter_eta
eventInternal ts_id                                       
0             558    23.049938  245.479675       -2.750928
              2242   16.149437  153.132401        2.673491
1             736    43.907665  138.009216       -1.773538
              2223   39.915539  155.163116        1.779360
2             757    19.182955   68.825623       -1.954545
...                        ...         ...             ...
97            2825   39.750000  226.456070        2.424081
98            749    62.088638  215.553238       -1.906013
              2349   54.445923  274.445282        1.895396
99            767    26.889877   96.964592       -1.901938
              2459   13.821290   79.518745        1.979664

[200 rows x 3 columns]

In [5]:
supercluster_df = reader.supercluster_df
supercluster_df

ts_id
eventInternal supercls_id ts_in_supercls_id       
0             0           0                    558
                          1                     89
              1           0                   2242
                          1                   2172
              2           0                    211
...                                            ...
99            268         1                   2156
              269         0                   2975
                          1                   3106
              270         0                   3044
                          1                   2170

[56735 rows x 1 columns]

In [28]:
tracksters[[]]

Empty DataFrame
Columns: []
Index: [(0, 558), (0, 2242), (1, 736), (1, 2223), (2, 757), (2, 2711), (3, 846), (3, 2302), (4, 562), (4, 2539), (5, 811), (5, 3422), (6, 926), (6, 2699), (7, 684), (7, 2503), (8, 1634), (8, 3345), (9, 817), (9, 3052), (10, 689), (10, 2282), (11, 504), (11, 2589), (12, 736), (12, 2105), (13, 605), (13, 1867), (14, 1912), (14, 4175), (15, 579), (15, 2382), (16, 529), (16, 2613), (17, 992), (17, 2878), (18, 448), (18, 2613), (19, 1212), (19, 1801), (20, 851), (20, 2869), (21, 885), (21, 2418), (22, 1230), (22, 2825), (23, 863), (23, 3024), (24, 656), (24, 2411), (25, 700), (25, 2797), (26, 1398), (26, 2508), (27, 698), (27, 3263), (28, 832), (28, 2398), (29, 730), (29, 2684), (30, 1002), (30, 2950), (31, 542), (31, 2089), (32, 783), (32, 2939), (33, 791), (33, 3019), (34, 1757), (34, 2407), (35, 718), (35, 2534), (36, 890), (36, 2320), (37, 742), (37, 2767), (38, 740), (38, 2537), (39, 772), (39, 2518), (40, 742), (40, 2453), (41, 708), (41, 2437), (42, 808), (42, 2358), (43, 562), (43, 2054), (44, 343), (44, 1976), (45, 909), (45, 2894), (46, 726), (46, 2625), (47, 767), (47, 2301), (48, 653), (48, 2639), (49, 736), (49, 2432), ...]

[200 rows x 0 columns]

In [26]:
supercluster_df.reset_index(["supercls_id", "ts_in_supercls_id"]).set_index("ts_id", append=True).convert_dtypes()

supercls_id  ts_in_supercls_id
eventInternal ts_id                                
0             558              0                  0
              89               0                  1
              2242             1                  0
              2172             1                  1
              211              2                  0
...                          ...                ...
99            2156           268                  1
              2975           269                  0
              3106           269                  1
              3044           270                  0
              2170           270                  1

[56735 rows x 2 columns]

In [21]:
merged_df = pd.merge(
    tracksters, 
    # use convert_dtypes to avoid converting ids to float when there are missing values
    supercluster_df.reset_index(["supercls_id", "ts_in_supercls_id"]).set_index("ts_id", append=True).convert_dtypes(),
    left_index=True, right_index=True,
    how="inner"
)
merged_df

raw_pt  raw_energy  barycenter_eta  supercls_id  \
eventInternal ts_id                                                       
0             558    23.049938  245.479675       -2.750928            0   
              2242   16.149437  153.132401        2.673491            1   
1             736    43.907665  138.009216       -1.773538            0   
              2223   39.915539  155.163116        1.779360            1   
2             757    19.182955   68.825623       -1.954545            1   
...                        ...         ...             ...          ...   
97            2825   39.750000  226.456070        2.424081            0   
98            749    62.088638  215.553238       -1.906013            0   
              2349   54.445923  274.445282        1.895396            1   
99            767    26.889877   96.964592       -1.901938            0   
              2459   13.821290   79.518745        1.979664            1   

                     ts_in_supercls_id  
eventInternal ts_id                     
0             558                    0  
              2242                   0  
1             736                    0  
              2223                   0  
2             757                    0  
...                                ...  
97            2825                   0  
98            749                    0  
              2349                   0  
99            767                    0  
              2459                   0  

[132 rows x 5 columns]

In [22]:
merged_step2 = pd.merge(merged_df, 
         supercluster_df,
        left_on=["eventInternal", "supercls_id"],
        right_on=["eventInternal", "supercls_id"],
        how="inner").set_index("ts_id", append=True)
merged_step2

raw_pt  raw_energy  barycenter_eta  supercls_id  \
eventInternal ts_id                                                       
0             558    23.049938  245.479675       -2.750928            0   
              89     23.049938  245.479675       -2.750928            0   
              2242   16.149437  153.132401        2.673491            1   
              2172   16.149437  153.132401        2.673491            1   
1             736    43.907665  138.009216       -1.773538            0   
...                        ...         ...             ...          ...   
99            1031   26.889877   96.964592       -1.901938            0   
              1172   26.889877   96.964592       -1.901938            0   
              2459   13.821290   79.518745        1.979664            1   
              2663   13.821290   79.518745        1.979664            1   
              2465   13.821290   79.518745        1.979664            1   

                     ts_in_supercls_id  
eventInternal ts_id                     
0             558                    0  
              89                     0  
              2242                   0  
              2172                   0  
1             736                    0  
...                                ...  
99            1031                   0  
              1172                   0  
              2459                   0  
              2663                   0  
              2465                   0  

[458 rows x 5 columns]

In [25]:
pd.merge(
        merged_step2, 
        tracksters_toDf(allTracksters),
        left_index=True, right_index=True,
        how="inner",
    )

raw_pt_x  raw_energy_x  barycenter_eta_x  supercls_id  \
eventInternal ts_id                                                           
0             558    23.049938    245.479675         -2.750928            0   
              89     23.049938    245.479675         -2.750928            0   
              2242   16.149437    153.132401          2.673491            1   
              2172   16.149437    153.132401          2.673491            1   
1             736    43.907665    138.009216         -1.773538            0   
...                        ...           ...               ...          ...   
99            1031   26.889877     96.964592         -1.901938            0   
              1172   26.889877     96.964592         -1.901938            0   
              2459   13.821290     79.518745          1.979664            1   
              2663   13.821290     79.518745          1.979664            1   
              2465   13.821290     79.518745          1.979664            1   

                     ts_in_supercls_id   raw_pt_y  raw_energy_y  \
eventInternal ts_id                                               
0             558                    0  23.049938    245.479675   
              89                     0   0.353635      1.591108   
              2242                   0  16.149437    153.132401   
              2172                   0   0.440235      2.142522   
1             736                    0  43.907665    138.009216   
...                                ...        ...           ...   
99            1031                   0   2.231263     16.328894   
              1172                   0   0.922971      2.901897   
              2459                   0  13.821290     79.518745   
              2663                   0  12.127513     60.812607   
              2465                   0   0.441317      2.950081   

                     barycenter_eta_y  
eventInternal ts_id                    
0             558           -2.750928  
              89            -2.716250  
              2242           2.673491  
              2172           2.652903  
1             736           -1.773538  
...                               ...  
99            1031          -1.803146  
              1172          -1.868977  
              2459           1.979664  
              2663           1.894073  
              2465           1.941481  

[458 rows x 8 columns]

In [35]:
pd.options.display.min_rows = 50
res = trackster_joinSupercluster((tracksters), tracksters_toDf(allTracksters), supercluster_df)
res

supercls_id  ts_in_supercls_id     raw_pt  raw_energy  \
eventInternal ts_id                                                          
0             558              0                  0  23.049938  245.479675   
              89               0                  0   0.353635    1.591108   
              2242             1                  0  16.149437  153.132401   
              2172             1                  0   0.440235    2.142522   
1             736              0                  0  43.907665  138.009216   
              452              0                  0   3.480924   10.312819   
              2223             1                  0  39.915539  155.163116   
              2101             1                  0  13.510190   44.060501   
2             757              1                  0  19.182955   68.825623   
              201              1                  0  10.293271   37.115944   
              649              1                  0   2.972689    8.562540   
              819              1                  0   0.975658    2.218431   
              2711             0                  0  29.842718  102.388222   
              2776             0                  0  25.053234   95.741249   
              3016             0                  0   2.144511   14.321663   
              2120             0                  0   0.395619    1.935641   
              2119             0                  0   0.392101    1.033118   
3             2302             0                  0  34.947758  246.589294   
              1742             0                  0   0.927618    3.292508   
4             2539             0                  0  22.151016  165.199677   
              2108             0                  0   0.638864    1.398566   
              2281             0                  0   0.448170    3.382849   
6             926              1                  0  40.198135  183.529160   
              972              1                  0  24.483976  137.773102   
              973              1                  0   0.513179    1.724289   
...                          ...                ...        ...         ...   
95            889              0                  0  11.048564   61.473553   
              95               0                  0   0.849373    3.392305   
96            1006             0                  0   9.845423   34.299637   
              812              0                  0   1.108697    6.831743   
              212              0                  0   0.395940    0.934254   
97            572              1                  0  25.325029  119.897163   
              683              1                  0   6.053311  104.547867   
              272              1                  0   0.538348    3.207276   
              2825             0                  0  39.750000  226.456070   
              2511             0                  0   0.409070    3.401215   
98            749              0                  0  62.088638  215.553238   
              1000             0                  0   3.343981   22.826790   
              803              0                  0   0.579779    3.142944   
              672              0                  0   0.521232    1.994047   
              2349             1                  0  54.445923  274.445282   
              2578             1                  0   4.031399   19.880714   
              2819             1                  0   3.137617    5.925182   
              1847             1                  0   1.294407    2.265998   
99            767              0                  0  26.889877   96.964592   
              695              0                  0   6.293355   24.021206   
              1031             0                  0   2.231263   16.328894   
              1172             0                  0   0.922971    2.901897   
              2459             1                  0  13.821290   79.518745   
              2663             1           

In [37]:
reader.assocs_bestScore_recoToSim_df()

simts_id     score    sharedE
eventInternal ts_id                               
0             0             0  0.995026   0.001160
              1             0  0.993735   0.001746
              2             0  0.765515   0.017806
              3             0  0.988891   0.003437
              4             0  0.994827   0.000830
              5             0  0.993625   0.078724
              6             0  0.436521   1.054007
              7             0  0.539568   0.308895
              8             0  0.977086   0.008888
              9             0  0.992916   0.002302
              10            0  0.983538   0.150715
              11            0  0.956263   0.001826
              12            0  0.983847   0.013902
              13            0  0.881846   0.020234
              14            0  0.926609   0.111788
              15            0  0.972505   0.004426
              16            0  0.998127   0.112023
              17            0  0.997324   0.000315
              18            0  0.991639   0.000231
              19            0  0.995330   0.001622
              20            0  0.997610   0.013759
              21            0  0.998681   0.003996
              22            0  0.993810   0.013299
              23            0  0.990355   0.000706
              24            0  0.795188   0.192712
...                       ...       ...        ...
99            48            1  0.989615   0.013312
              49            1  0.997003   0.007489
              50            1  0.998348   0.000471
              51            1  0.908948   0.344634
              52            1  0.058458   3.930435
              53            1  0.995811   0.001299
              54            1  0.989815   0.028945
              55            1  0.000713  73.737274
              56            1  0.014660   2.345443
              57            1  0.999429   0.000695
              58            1  0.001095   9.409070
              59            1  0.880621   0.004346
              60            1  0.999853   0.001189
              61            1  0.000125  43.663654
              62            1  0.001236  56.770634
              63            1  0.999817   0.001648
              64            1  0.344174   2.068451
              65            1  0.643722   1.634701
              66            1  0.684409   0.409644
              67            1  0.006426   0.610563
              68            1  0.000253   2.993102
              69            1  0.991708   0.002847
              70            1  0.000000   2.490121
              71            1  0.000603   1.619693
              72            1  0.879007   0.003743

[9314 rows x 3 columns]

In [40]:
res.join(reader.assocs_bestScore_recoToSim_df())

supercls_id  ts_in_supercls_id     raw_pt  raw_energy  \
eventInternal ts_id                                                          
0             558              0                  0  23.049938  245.479675   
              89               0                  0   0.353635    1.591108   
              2242             1                  0  16.149437  153.132401   
              2172             1                  0   0.440235    2.142522   
1             736              0                  0  43.907665  138.009216   
              452              0                  0   3.480924   10.312819   
              2223             1                  0  39.915539  155.163116   
              2101             1                  0  13.510190   44.060501   
2             757              1                  0  19.182955   68.825623   
              201              1                  0  10.293271   37.115944   
              649              1                  0   2.972689    8.562540   
              819              1                  0   0.975658    2.218431   
              2711             0                  0  29.842718  102.388222   
              2776             0                  0  25.053234   95.741249   
              3016             0                  0   2.144511   14.321663   
              2120             0                  0   0.395619    1.935641   
              2119             0                  0   0.392101    1.033118   
3             2302             0                  0  34.947758  246.589294   
              1742             0                  0   0.927618    3.292508   
4             2539             0                  0  22.151016  165.199677   
              2108             0                  0   0.638864    1.398566   
              2281             0                  0   0.448170    3.382849   
6             926              1                  0  40.198135  183.529160   
              972              1                  0  24.483976  137.773102   
              973              1                  0   0.513179    1.724289   
...                          ...                ...        ...         ...   
95            889              0                  0  11.048564   61.473553   
              95               0                  0   0.849373    3.392305   
96            1006             0                  0   9.845423   34.299637   
              812              0                  0   1.108697    6.831743   
              212              0                  0   0.395940    0.934254   
97            572              1                  0  25.325029  119.897163   
              683              1                  0   6.053311  104.547867   
              272              1                  0   0.538348    3.207276   
              2825             0                  0  39.750000  226.456070   
              2511             0                  0   0.409070    3.401215   
98            749              0                  0  62.088638  215.553238   
              1000             0                  0   3.343981   22.826790   
              803              0                  0   0.579779    3.142944   
              672              0                  0   0.521232    1.994047   
              2349             1                  0  54.445923  274.445282   
              2578             1                  0   4.031399   19.880714   
              2819             1                  0   3.137617    5.925182   
              1847             1                  0   1.294407    2.265998   
99            767              0                  0  26.889877   96.964592   
              695              0                  0   6.293355   24.021206   
              1031             0                  0   2.231263   16.328894   
              1172             0                  0   0.922971    2.901897   
              2459             1                  0  13.821290   79.518745   
              2663             1           

In [44]:
ak.Array([[[[]]]]).ndim

4